## 参考资料

中国科学院软件研究所 刘焕勇老师 https://github.com/liuhuanyong/QASystemOnMedicalKG

## 导入工具包

In [1]:
import os
import json
from py2neo import Graph, Node

import pandas as pd

## 导入数据集csv文件

In [2]:
df = pd.read_csv('medical_data.csv')

In [3]:
df.shape

(21, 23)

## 实体（节点）

### 实体：所有症状

In [4]:
symptoms = []
for each in df['症状']:
    symptoms.extend(each.split(','))
symptoms = set(symptoms)

### 所有科室

In [5]:
departments = []
for each in df['科室']:
    departments.extend(each.split(','))
departments = set(departments)

### 实体：所有检查

In [6]:
checks = []
for each in df['检查']:
    checks.extend(each.split(','))
checks = set(checks)

### 实体：所有药物

In [7]:
drugs = []
for each in df['推荐药物']:
    try:
        drugs.extend(each.split(','))
    except:
        pass
for each in df['常用药物']:
    try:
        drugs.extend(each.split(','))
    except:
        pass
drugs = set(drugs)

### 实体：所有食物

In [8]:
foods = []
for each in df['可以吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
for each in df['不可以吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
for each in df['推荐吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
foods = set(foods)

### 实体：所有药物厂商

In [9]:
producers = []

for each in df['具体药物']:
    try:
        for each_drug in each.split(','):
            producer = each_drug.split('(')[0]
            producers.append(producer)
    except:
        pass
producers = set(producers)

### 疾病字典信息

In [10]:
disease_infos = [] # 疾病信息
for idx, row in df.iterrows():
    disease_infos.append(dict(row))

In [11]:
dict(row).keys()

dict_keys(['疾病名称', '疾病描述', '疾病种类', '科室', '病因', '症状', '检查', '并发症', '花费', '疗程', '疗法', '治愈率', '易感人群', '感染概率', '感染途径', '预防措施', '推荐药物', '常用药物', '具体药物', '可以吃', '不可以吃', '推荐吃', '是否纳入医保'])

## 关系（边）

In [29]:
def deduplicate_list(rels_old):
    '''关系去重函数: list'''
    rels_new = []
    for each in rels_old:
        if each not in rels_new:
            rels_new.append(each)
    return rels_new

### 关系：疾病-检查

In [30]:
rels_check = []
for idx, row in df.iterrows():
    for each in row['检查'].split(','):
        rels_check.append([row['疾病名称'], each])
rels_check = deduplicate_list(rels_check)

In [33]:
rels_check[:4]

[['肺泡蛋白质沉积症', '胸部CT检查'],
 ['肺泡蛋白质沉积症', '肺活检'],
 ['肺泡蛋白质沉积症', '支气管镜检查'],
 ['百日咳', '耳、鼻、咽拭子细菌培养']]

### 关系：疾病-症状

In [15]:
rels_symptom = []
for idx, row in df.iterrows():
    for each in row['症状'].split(','):
        rels_symptom.append([row['疾病名称'], each])
rels_symptom = deduplicate_list(rels_symptom)

In [34]:
rels_symptom[:4]

[['肺泡蛋白质沉积症', '紫绀'],
 ['肺泡蛋白质沉积症', '胸痛'],
 ['肺泡蛋白质沉积症', '呼吸困难'],
 ['肺泡蛋白质沉积症', '乏力']]

### 关系：疾病-疾病（并发症）

In [17]:
rels_acompany = []
for idx, row in df.iterrows():
    for each in row['并发症'].split(','):
        rels_acompany.append([row['疾病名称'], each])
rels_acompany = deduplicate_list(rels_acompany)

In [36]:
rels_acompany[:10]

[['肺泡蛋白质沉积症', '多重肺部感染'],
 ['百日咳', '肺不张'],
 ['苯中毒', '贫血'],
 ['喘息样支气管炎', '支气管哮喘'],
 ['成人呼吸窘迫综合征', '细菌性肺炎'],
 ['大量羊水吸入', '呼吸衰竭'],
 ['单纯性肺嗜酸粒细胞浸润症', '胆道蛔虫病'],
 ['大叶性肺炎', '脓胸'],
 ['大楼病综合征', '抑郁症'],
 ['二硫化碳中毒', '昏迷']]

### 关系：疾病-推荐药物

In [19]:
rels_recommanddrug = []
for idx, row in df.iterrows():
    try:
        for each in row['推荐药物'].split(','):
            rels_recommanddrug.append([row['疾病名称'], each])
    except:
        pass
rels_recommanddrug = deduplicate_list(rels_recommanddrug)

### 关系：疾病-常用药物

In [20]:
rels_commonddrug = []
for idx, row in df.iterrows():
    try:
        for each in row['常用药物'].split(','):
            rels_commonddrug.append([row['疾病名称'], each])
    except:
        pass
rels_commonddrug = deduplicate_list(rels_commonddrug)

### 关系：疾病-不可以吃

In [21]:
rels_noteat = []
for idx, row in df.iterrows():
    try:
        for each in row['不可以吃'].split(','):
            rels_noteat.append([row['疾病名称'], each])
    except:
        pass
rels_noteat = deduplicate_list(rels_noteat)

### 关系：疾病-可以吃

In [22]:
rels_doeat = []
for idx, row in df.iterrows():
    try:
        for each in row['可以吃'].split(','):
            rels_doeat.append([row['疾病名称'], each])
    except:
        pass
rels_doeat = deduplicate_list(rels_doeat)

### 关系：疾病-推荐吃

In [23]:
rels_recommandeat = []
for idx, row in df.iterrows():
    try:
        for each in row['推荐吃'].split(','):
            rels_recommandeat.append([row['疾病名称'], each])
    except:
        pass
rels_recommandeat = deduplicate_list(rels_recommandeat)

### 关系：药物厂商-具体药物

In [24]:
rels_drug_producer = []
for each in df['具体药物']:
    try:
        for each_drug in each.split(','):
            producer = each_drug.split('(')[0]
            drug = each_drug.split('(')[1][:-1]
            rels_drug_producer.append([producer, drug])
    except:
        pass
rels_drug_producer = deduplicate_list(rels_drug_producer)

### 关系：疾病-科室、小科室-大科室

In [25]:
rels_category = [] # 关系：疾病-科室
rels_department = [] # 关系：小科室-大科室
for idx, row in df.iterrows():
    if len(row['科室'].split(',')) == 1:
        rels_category.append([row['疾病名称'], row['科室']])
    else:
        big = row['科室'].split(',')[0] # 大科室
        small = row['科室'].split(',')[1] # 小科室
        rels_category.append([row['疾病名称'], small])
        rels_department.append([small, big])
rels_category = deduplicate_list(rels_category)
rels_department = deduplicate_list(rels_department)

In [37]:
rels_category[:10]

[['肺泡蛋白质沉积症', '呼吸内科'],
 ['百日咳', '小儿内科'],
 ['苯中毒', '急诊科'],
 ['喘息样支气管炎', '呼吸内科'],
 ['成人呼吸窘迫综合征', '呼吸内科'],
 ['大量羊水吸入', '小儿内科'],
 ['单纯性肺嗜酸粒细胞浸润症', '呼吸内科'],
 ['大叶性肺炎', '呼吸内科'],
 ['大楼病综合征', '其他综合'],
 ['二硫化碳中毒', '急诊科']]

In [38]:
rels_department

[['呼吸内科', '内科'], ['小儿内科', '儿科'], ['其他综合', '其他科室']]

## 连接图数据库

In [39]:
# 注意，这里的用户名为neo4j全局用户名，而非DBMS或者database的名称
g = Graph('http://localhost:7474', auth=('neo4j', 'medical_neo4j'), name="neo4j")

## 创建知识图谱实体（节点）

### 样例代码

In [40]:
# # Neo4j样例代码

# 创建实体
# node = Node('Disease', name='百日咳', easy_get='多见于小儿')
# g.create(node)

In [41]:
# 删除所有实体和关系
cypher = 'MATCH (n) DETACH DELETE n'
g.run(cypher)

n


### 创建疾病实体

In [51]:
count = 0
for disease_dict in disease_infos:
    try:
        node = Node("Disease",
                    name=disease_dict['疾病名称'],
                    desc=disease_dict['疾病描述'],
                    prevent=disease_dict['预防措施'],
                    cause=disease_dict['病因'],
                    easy_get=disease_dict['易感人群'],
                    cure_lasttime=disease_dict['疗程'],
                    cure_department=disease_dict['科室'],
                    cure_way=disease_dict['疗法'], 
                    cured_prob=disease_dict['治愈率'])
        g.create(node)
        count += 1
        print('创建疾病实体：', disease_dict['疾病名称'])
    except:
        print(f"创建失败 Disease {disease_dict['疾病名称']}")
        pass
print('共创建 {} 个疾病实体'.format(count))

创建失败 Disease 肺泡蛋白质沉积症
创建疾病实体： 百日咳
创建疾病实体： 苯中毒
创建疾病实体： 喘息样支气管炎
创建疾病实体： 成人呼吸窘迫综合征
创建疾病实体： 大量羊水吸入
创建疾病实体： 单纯性肺嗜酸粒细胞浸润症
创建疾病实体： 大叶性肺炎
创建疾病实体： 大楼病综合征
创建疾病实体： 二硫化碳中毒
创建疾病实体： 肺-胸膜阿米巴病
创建疾病实体： 肺出血－肾炎综合征
创建疾病实体： 肺放线菌病
创建疾病实体： 肺泡蛋白沉着症
创建疾病实体： 肺曲菌病
创建疾病实体： 放射性肺炎
创建疾病实体： 肺念珠菌病
创建疾病实体： 肺大疱
创建疾病实体： 肺炎球菌肺炎
创建疾病实体： 肺气肿
创建疾病实体： 肺炎杆菌肺炎
共创建 20 个疾病实体


### 创建药物实体

In [52]:
for each in drugs:
    node = Node('Drug', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 盐酸氨溴索葡萄糖注射液
创建实体 仁青芒觉
创建实体 琥乙红霉素片
创建实体 蒺藜药酒
创建实体 甲硝唑维B6片
创建实体 硝酸益康唑溶液
创建实体 蛤蚧定喘丸
创建实体 硝酸咪康唑乳膏
创建实体 注射用美洛西林钠舒巴坦钠
创建实体 替硝唑片
创建实体 盐酸左氧氟沙星胶囊
创建实体 注射用盐酸头孢替安
创建实体 头孢克肟分散片
创建实体 氨苄西林胶囊
创建实体 穿心莲内酯片
创建实体 蛇胆川贝液
创建实体 枸橼酸喷托维林片
创建实体 头孢拉定胶囊
创建实体 头孢氨苄甲氧苄啶片
创建实体 硫酸特布他林雾化液
创建实体 胸腺肽肠溶片
创建实体 注射用硫代硫酸钠
创建实体 盐酸班布特罗胶囊
创建实体 托西酸舒他西林胶囊
创建实体 十一味金色丸
创建实体 诺氟沙星胶囊
创建实体 地塞米松磷酸钠注射液
创建实体 注射用盐酸溴己新
创建实体 环吡酮胺乳膏
创建实体 联苯苄唑乳膏
创建实体 注射用甲泼尼龙琥珀酸钠
创建实体 头孢克肟颗粒
创建实体 氨茶碱片
创建实体 布美他尼片
创建实体 噻托溴铵粉吸入剂
创建实体 头孢克洛颗粒
创建实体 头孢拉定分散片
创建实体 化痰平喘片
创建实体 硫酸沙丁胺醇气雾剂
创建实体 通宣理肺丸
创建实体 硫酸沙丁胺醇片
创建实体 甲泼尼龙片
创建实体 雷公藤多苷片
创建实体 二羟丙茶碱片
创建实体 阿奇霉素胶囊
创建实体 红霉素肠溶片
创建实体 注射用还原型谷胱甘肽钠
创建实体 环酯红霉素片
创建实体 奥硝唑片
创建实体 硫酸特布他林片
创建实体 小青龙颗粒
创建实体 茶碱缓释胶囊
创建实体 白及颗粒
创建实体 复方蛤青胶囊
创建实体 大月晶丸
创建实体 阿奇霉素颗粒
创建实体 氟康唑胶囊
创建实体 依托红霉素片
创建实体 注射用头孢唑林钠
创建实体 司帕沙星分散片
创建实体 克霉唑阴道片
创建实体 甲硝唑片
创建实体 头孢丙烯分散片
创建实体 盐酸克林霉素胶囊
创建实体 注射用呋塞米
创建实体 硝酸咪康唑栓
创建实体 注射用布美他尼
创建实体 盐酸妥洛特罗片
创建实体 布地奈德气雾剂
创建实体 盐酸利多卡因注射液
创建实体 百咳静糖浆
创建实体 琥乙红霉素颗粒
创建实体 硝酸益康唑乳膏
创建实体 茶碱缓释片
创建实体 阿奇霉素片
创建实体 麦迪霉素片
创建实体 阿奇霉素分散片
创建实体 云南花粉片
创建实体 盐酸班布特罗片
创建实体 

### 创建食物实体

In [53]:
for each in foods:
    node = Node('Food', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 番茄牛肉片
创建实体 鲈鱼
创建实体 海蟹
创建实体 山楂桑椹粥
创建实体 生菜
创建实体 豆薯小米粥
创建实体 奶汤锅子鱼
创建实体 三鲜冬瓜汤
创建实体 番茄芹菜汁
创建实体 瓜粒杂锦汤
创建实体 蔬菜杂烩粥
创建实体 鸡蛋木耳粥
创建实体 南瓜子仁
创建实体 罗汉果雪耳鸡汤
创建实体 绿豆南瓜汤
创建实体 松子仁
创建实体 辣椒(青、尖)
创建实体 番茄牛肉
创建实体 羊肉(瘦)
创建实体 意式番茄牛肉凉面
创建实体 冬瓜草鱼汤
创建实体 栗子（熟）
创建实体 咖啡
创建实体 海蚌
创建实体 竹笋
创建实体 鸭梨粥
创建实体 小黄瓜西瓜汁
创建实体 白酒
创建实体 番茄汁
创建实体 素炒小白菜
创建实体 鲢鱼
创建实体 花生杏仁粥
创建实体 茭白
创建实体 啤酒
创建实体 大豆
创建实体 鲤鱼
创建实体 冬菇烧白菜
创建实体 荞麦粥
创建实体 百合炒芦笋
创建实体 拌肚丝白菜
创建实体 银耳粥
创建实体 海螺
创建实体 党参粥
创建实体 五丝白菜卷
创建实体 山楂红糖粥
创建实体 番茄猪肝瘦肉汤
创建实体 川贝酿梨
创建实体 豆腐干炒韭菜
创建实体 辣椒(红、尖、干)
创建实体 黄瓜拌兔丝
创建实体 黄瓜拌豆腐丝
创建实体 油麦菜
创建实体 螃蟹
创建实体 百合炖猪蹄
创建实体 糖醋草鱼
创建实体 豆浆
创建实体 雪梨饮
创建实体 鹿肉
创建实体 红枣银耳粥
创建实体 大蒜
创建实体 锅塌番茄
创建实体 小白菜
创建实体 山楂百合汤
创建实体 百合粥
创建实体 鲫鱼
创建实体 山药
创建实体 苦瓜
创建实体 鸡肝
创建实体 冰糖银耳羹
创建实体 炖白鲢鱼
创建实体 羊腩萝卜丝粥
创建实体 鱼腥草
创建实体 圆白菜
创建实体 泡莲藕
创建实体 海虾
创建实体 百合双耳鸡蛋羹
创建实体 番茄鸡蛋煎饼
创建实体 茄子
创建实体 凉拌番茄
创建实体 赤贝
创建实体 牛奶玉米汤
创建实体 油菜
创建实体 空心菜
创建实体 酱豆腐汁烧猪肉
创建实体 百合汁
创建实体 番茄羊肉汤
创建实体 核桃
创建实体 莲子
创建实体 北杏炖雪梨
创建实体 凉拌莲藕
创建实体 鲜百合烧肉
创建实体 黄芪粥
创建实体 水萝卜丝拌白菜
创建实体 珊瑚白菜墩
创建实体 番茄羊肉蛋花汤
创建实体 黄瓜拌香菜
创建实体 百合汤
创建实体 鸡蛋
创建实体 番茄三素
创建实体 玉竹白菜
创建

### 创建检查实体

In [54]:
for each in checks:
    node = Node('Check', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 血常规
创建实体 肺活量（VC）
创建实体 支气管造影
创建实体 Optochin敏感试验
创建实体 呼吸肌功能测定
创建实体 残气量／肺总量比值（RV/TLC）
创建实体 无效腔气量／潮气量比值
创建实体 纤维支气管镜检查
创建实体 痰液中寄生虫和虫卵
创建实体 周围血白细胞计数及分类检验
创建实体 钼靶X线检查
创建实体 免疫电泳
创建实体 一秒用力呼出量／用力肺活量比值
创建实体 尿液镜检法
创建实体 痰液中细胞分类
创建实体 肺泡气-动脉血氧分压差
创建实体 抗肾小球基底膜抗体测定（AGBM）
创建实体 肺活检
创建实体 胸部CT检查
创建实体 通气与血流灌注比值（V/Q）
创建实体 骨髓象分析
创建实体 白细胞分类计数
创建实体 每分钟最大通气量（MVV）
创建实体 酶联免疫吸附试验
创建实体 脑电图检查
创建实体 最大呼气流量-容积曲线（MEFV）
创建实体 肺部检查
创建实体 隐血试验与含铁血黄素检查
创建实体 肺功能检查
创建实体 痰液细菌涂片检查
创建实体 痰液病原体检查
创建实体 深吸气量（IC）
创建实体 胸部平片
创建实体 尿常规
创建实体 支气管镜检查
创建实体 先令氏指数
创建实体 耳、鼻、咽拭子细菌培养
创建实体 肺活量体重指数
创建实体 小白鼠毒力试验
创建实体 神经系统检查
创建实体 CT检查
创建实体 肺毛细血管楔压
创建实体 痰培养
创建实体 肺和胸膜听诊
创建实体 眼底荧光血管造影
创建实体 心肺功能运动试验（CPET）
创建实体 肺容量测定
创建实体 肺量计检测
创建实体 肌电图
创建实体 痰液常规检查
创建实体 心电图
创建实体 肾功能检查
创建实体 抗链球菌壁多糖抗体
创建实体 血浆蛋白C抗原
创建实体 抗链球菌型M蛋白抗体
创建实体 痰液细菌培养
创建实体 胸部透视


### 创建科室实体

In [55]:
for each in departments:
    node = Node('Department', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 其他科室
创建实体 急诊科
创建实体 其他综合
创建实体 小儿内科
创建实体 内科
创建实体 呼吸内科
创建实体 儿科


### 创建 药物厂商 实体

In [56]:
for each in producers:
    node = Node('Producer', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 重庆华邦制药注射用甲泼尼龙
创建实体 黄河药业盐酸克林霉素胶囊
创建实体 珠海联邦头孢拉定胶囊
创建实体 衡博来
创建实体 澳利达奈德注射用盐酸溴己新
创建实体 东药阿奇霉素胶囊
创建实体 纽兰药业小青龙合剂
创建实体 七星山药业化痰平喘片
创建实体 鲁抗阿莫西林颗粒
创建实体 哈药总厂注射用磺苄西林钠
创建实体 德远药业雷公藤多苷片
创建实体 佐今明百咳静糖浆
创建实体 金路捷
创建实体 丽珠奇乐
创建实体 东信药业琥乙红霉素颗粒
创建实体 金诃藏药仁青芒觉
创建实体 智同生物注射用盐酸溴己新
创建实体 康正药业硝酸咪康唑乳膏
创建实体 亮博
创建实体 潇然
创建实体 宜昌人福盐酸克林霉素胶囊
创建实体 比利时辉瑞注射用甲泼尼龙琥
创建实体 旺龙药业琥乙红霉素颗粒
创建实体 苏州三药盐酸克林霉素胶囊
创建实体 恒安药业盐酸特比萘芬片
创建实体 泰邦生物人血白蛋白
创建实体 桑海制药喷托维林氯化铵糖浆
创建实体 北京中新制药琥乙红霉素片
创建实体 遂成药业地塞米松磷酸钠注射
创建实体 埃斯玛隆
创建实体 仁和药业盐酸克林霉素胶囊
创建实体 全星制药注射用布美他尼
创建实体 南通中华药业硝酸咪康唑乳膏
创建实体 思力华
创建实体 湖南九典制药盐酸班布特罗胶
创建实体 康裕头孢氨苄胶囊
创建实体 米乐松
创建实体 永定制药百咳静糖浆
创建实体 天天乐药业蛇胆川贝液
创建实体 哈药总厂托西酸舒他西林胶囊
创建实体 三维康
创建实体 润阳药业盐酸氨溴索葡萄糖注
创建实体 傲宁
创建实体 申优
创建实体 仁和堂硫酸沙丁胺醇片
创建实体 桂林华信环吡酮胺乳膏
创建实体 斯皮仁诺
创建实体 罗欣药业注射用盐酸氨溴索
创建实体 达仁堂通宣理肺丸
创建实体 注射用盐酸头孢替安
创建实体 吉尔吉药业喘嗽宁片
创建实体 博利康尼
创建实体 易坦静
创建实体 津华晖星甲硝唑片
创建实体 德源生物人血白蛋白
创建实体 白云山光华头孢氨苄甲氧苄啶
创建实体 纽兰药业硝酸咪康唑栓
创建实体 爱普森药业阿奇霉素片
创建实体 吉林东方制药枸橼酸喷托维林
创建实体 西岳制药托西酸舒他西林胶囊
创建实体 佳乐弗乳酸左氧氟沙星片
创建实体 鲁神乙酰螺旋霉素片
创建实体 西藏甘露仁青芒觉
创建实体 北京京丰制药集团诺氟沙星胶
创建实体 海南制药厂蛇胆川贝液
创建实体 莎普爱思盐酸氨溴索葡萄糖注
创

### 创建 症状 实体

In [57]:
for each in symptoms:
    node = Node('Symptom', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 低热
创建实体 吸气时有蝉鸣音
创建实体 冬春季的慢性咳...
创建实体 骨膜炎
创建实体 气急
创建实体 耸肩喘息
创建实体 呕吐
创建实体 小支气管粘膜水肿
创建实体 出生后即有持续青紫
创建实体 蛋白尿
创建实体 咳出棕色痰栓
创建实体 胸痛
创建实体 横膈低平
创建实体 乏力
创建实体 恶心
创建实体 胸闷憋气
创建实体 腱反射消失
创建实体 口唇和甲床略带青紫
创建实体 纤毛上皮细胞损伤脱落
创建实体 呼吸音减弱
创建实体 喘息
创建实体 闫鹏辉
创建实体 眼睛痒
创建实体 肺纤维化
创建实体 发烧
创建实体 腹泻
创建实体 黏稠或脓性痰伴...
创建实体 口唇青紫
创建实体 气短
创建实体 发热伴咳嗽、咯...
创建实体 体重减轻
创建实体 咳嗽伴胸痛
创建实体 肺部啰音
创建实体 昏迷
创建实体 紧张性头晕
创建实体 谵妄
创建实体 浅感觉减退或缺失
创建实体 桶状胸
创建实体 胸闷
创建实体 惊厥
创建实体 面色青紫
创建实体 化学性支气管炎
创建实体 湿啰音
创建实体 盗汗
创建实体 咯血伴发热
创建实体 咳铁锈色痰
创建实体 紫绀
创建实体 间歇性头晕
创建实体 肺泡灌洗液可见...
创建实体 呼气音延长
创建实体 闫铁
创建实体 纵隔浊音界扩大
创建实体 咯血
创建实体 感觉障碍
创建实体 痰呈粘液脓性
创建实体 多发性神经炎
创建实体 肺阴虚
创建实体 肺纹理增粗
创建实体 头晕
创建实体 痰中带血丝
创建实体 呼吸困难
创建实体 嗜酸性粒细胞增多
创建实体 肺泡炎症
创建实体 鼻塞
创建实体 哮鸣音
创建实体 紧张性头痛
创建实体 抽搐
创建实体 畏寒
创建实体 痉挛性咳嗽
创建实体 急性面容
创建实体 咽部异物感
创建实体 间歇性头痛
创建实体 目赤
创建实体 心源性呼吸窘迫
创建实体 变应性咳嗽
创建实体 毓卓
创建实体 肺部出血
创建实体 咳嗽伴哮鸣音


## 创建知识图谱关系（连接、边）

### 样例代码

In [58]:
# # Neo4j样例代码

# # 删除所有实体和关系
# cypher = 'MATCH (n) DETACH DELETE n'
# g.run(cypher)

In [59]:
# # 创建关系 样例代码
# start_node = 'Disease'
# end_node = 'Check'
# p = '百日咳'
# q = '血常规'
# rel_type = 'need_check'
# rel_name = '诊断检查'

# # Cypher语句
# query = "match(p:%s),(q:%s) where p.name='%s' and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (start_node, end_node, p, q, rel_type, rel_name)
# print(query)
# g.run(query) # 运行 Cypher 语句


In [60]:
def create_relationship(start_node, end_node, edges, rel_type, rel_name):
    '''创建关系函数'''
    for edge in edges:
        p = edge[0]
        q = edge[1]
        # 创建关系的 Cypher 语句
        query = "match(p:%s),(q:%s) where p.name='%s' and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (start_node, end_node, p, q, rel_type, rel_name)
        try:
            g.run(query) # 运行 Cypher 语句
            print('创建关系 {}-{}->{}'.format(p, rel_type, q))
        except Exception as e:
            print(e)

### 创建所有关系

In [61]:
create_relationship('Disease', 'Food', rels_recommandeat, 'recommand_eat', '推荐食谱')
create_relationship('Disease', 'Food', rels_noteat, 'no_eat', '忌吃')
create_relationship('Disease', 'Food', rels_doeat, 'do_eat', '宜吃')
create_relationship('Department', 'Department', rels_department, 'belongs_to', '属于')
create_relationship('Disease', 'Drug', rels_commonddrug, 'common_drug', '常用药品')
create_relationship('Producer', 'Drug', rels_drug_producer, 'drugs_of', '生产药品')
create_relationship('Disease', 'Drug', rels_recommanddrug, 'recommand_drug', '好评药品')
create_relationship('Disease', 'Check', rels_check, 'need_check', '诊断检查')
create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom', '症状')
create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with', '并发症')
create_relationship('Disease', 'Department', rels_category, 'belongs_to', '所属科室')

创建关系 百日咳-recommand_eat->清蒸鸡蛋羹
创建关系 百日咳-recommand_eat->百合双耳鸡蛋羹
创建关系 百日咳-recommand_eat->排骨汤
创建关系 百日咳-recommand_eat->罗汉果雪耳鸡汤
创建关系 百日咳-recommand_eat->小黄瓜凉拌面
创建关系 百日咳-recommand_eat->黄瓜三丝汤
创建关系 百日咳-recommand_eat->黄瓜拌兔丝
创建关系 百日咳-recommand_eat->黄瓜拌皮丝
创建关系 苯中毒-recommand_eat->豆腐干炒韭菜
创建关系 苯中毒-recommand_eat->素炒小白菜
创建关系 苯中毒-recommand_eat->白菜蛋花粥
创建关系 喘息样支气管炎-recommand_eat->紫苏粥
创建关系 喘息样支气管炎-recommand_eat->菊花芦根茶
创建关系 喘息样支气管炎-recommand_eat->生芦根粥
创建关系 喘息样支气管炎-recommand_eat->鲜芦根粥
创建关系 喘息样支气管炎-recommand_eat->薏仁党参粥
创建关系 喘息样支气管炎-recommand_eat->党参粥
创建关系 喘息样支气管炎-recommand_eat->黄芪粥
创建关系 喘息样支气管炎-recommand_eat->鸡蛋木耳粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合糖粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合花生汤
创建关系 成人呼吸窘迫综合征-recommand_eat->首乌百合粥
创建关系 成人呼吸窘迫综合征-recommand_eat->薏米莲子粥
创建关系 成人呼吸窘迫综合征-recommand_eat->牛奶玉米汤
创建关系 成人呼吸窘迫综合征-recommand_eat->萝卜豆腐汤
创建关系 成人呼吸窘迫综合征-recommand_eat->瓜粒杂锦汤
创建关系 大叶性肺炎-recommand_eat->奶汤锅子鱼
创建关系 大叶性肺炎-recommand_eat->酱豆腐汁烧猪肉
创建关系 大叶性肺炎-recommand_eat->百合汤
创建关系 大叶性肺炎-recommand_